### Домашнє завдання: Пониження розмірностей для Аналізу Портретів Клієнтів

#### Контекст
В цьому ДЗ ми попрацюємо з методами пониження розмірності на наборі даних для задачі аналізу портретів клієнтів (Customer Personality Analysis). **В попередньому ДЗ ми працювали з цими даними використовуючи кластеризацію, зараз використаємо кластеризацію і візуалізауємо результати з різними методами.**

Customer Personality Analysis - це аналіз різних сегментів клієнтів компанії. Цей аналіз дозволяє бізнесу краще розуміти своїх клієнтів і полегшує процес адаптації продуктів під конкретні потреби, поведінку та інтереси різних типів клієнтів.

Аналіз портретів клієнтів допомагає бізнесу змінювати свій продукт на основі цільової аудиторії, розділеної на різні сегменти. Наприклад, замість того, щоб витрачати гроші на маркетинг нового продукту для всіх клієнтів у базі даних компанії, бізнес може проаналізувати, який сегмент клієнтів найімовірніше придбає продукт, і потім зосередити маркетингові зусилля лише на цьому сегменті.

#### Вхідні дані
Вам надано набір даних з такими атрибутами:

**Характеристики користувачів:**
- `ID`: Унікальний ідентифікатор клієнта
- `Year_Birth`: Рік народження клієнта
- `Education`: Рівень освіти клієнта
- `Marital_Status`: Сімейний стан клієнта
- `Income`: Річний дохід домогосподарства клієнта
- `Kidhome`: Кількість дітей у домогосподарстві клієнта
- `Teenhome`: Кількість підлітків у домогосподарстві клієнта
- `Dt_Customer`: Дата реєстрації клієнта у компанії
- `Recency`: Кількість днів з моменту останньої покупки клієнта
- `Complain`: 1, якщо клієнт скаржився за останні 2 роки, 0 - якщо ні

**Продукти:**
- `MntWines`: Сума, витрачена на вино за останні 2 роки
- `MntFruits`: Сума, витрачена на фрукти за останні 2 роки
- `MntMeatProducts`: Сума, витрачена на м'ясні продукти за останні 2 роки
- `MntFishProducts`: Сума, витрачена на рибні продукти за останні 2 роки
- `MntSweetProducts`: Сума, витрачена на солодощі за останні 2 роки
- `MntGoldProds`: Сума, витрачена на золото за останні 2 роки

**Акції:**
- `NumDealsPurchases`: Кількість покупок, зроблених з використанням знижок
- `AcceptedCmp1`: 1, якщо клієнт прийняв пропозицію у першій кампанії, 0 - якщо ні
- `AcceptedCmp2`: 1, якщо клієнт прийняв пропозицію у другій кампанії, 0 - якщо ні
- `AcceptedCmp3`: 1, якщо клієнт прийняв пропозицію у третій кампанії, 0 - якщо ні
- `AcceptedCmp4`: 1, якщо клієнт прийняв пропозицію у четвертій кампанії, 0 - якщо ні
- `AcceptedCmp5`: 1, якщо клієнт прийняв пропозицію у п'ятій кампанії, 0 - якщо ні
- `Response`: 1, якщо клієнт прийняв пропозицію в останній кампанії, 0 - якщо ні

**Взаємодія з компанією:**
- `NumWebPurchases`: Кількість покупок, зроблених через вебсайт компанії
- `NumCatalogPurchases`: Кількість покупок, зроблених за каталогом
- `NumStorePurchases`: Кількість покупок, зроблених безпосередньо у магазинах
- `NumWebVisitsMonth`: Кількість відвідувань вебсайту компанії за останній місяць


Для початку, запустіть код нижче. Всі ці кроки ми робили в попередньому ДЗ і для того, щоб результати кластеризації у нас були схожими, потрібно аби передобробка була однаковою.

In [1]:
import pandas as pd

# 1. Завантаження даних
df = pd.read_csv('marketing_campaign.csv', sep='\t')

# 2. Обробка пропущених значень
df['Income_not_filled'] = df.Income.isna()
df.Income = df.Income.fillna(-1)

# 3. Обробка дати реєстрації
df.Dt_Customer = pd.to_datetime(df.Dt_Customer, format='%d-%m-%Y')
today = df.Dt_Customer.max()
df['days_lifetime'] = (today - df.Dt_Customer).dt.days
df['years_customer'] = df.Year_Birth.apply(lambda x: today.year - x)

# 4. Категоризація рівня освіти
df_education = pd.get_dummies(df.Education, prefix='education').astype(int)
df = pd.concat([df, df_education], axis=1)

# 5. Очищення сімейного стану
marital_status_map = {'Alone': 'Single', 'Absurd': 'Else', 'YOLO': 'Else'}
df['Marital_Status_clean'] = df.Marital_Status.map(marital_status_map)
df_ms = pd.get_dummies(df.Marital_Status_clean, prefix='marital').astype(int)
df = pd.concat([df, df_ms], axis=1)

# 6. Форматування доходу і видалення викиду
df.Income = df.Income.astype(int)
df = df[df.Income != 666666]

# 7. Створення фінального набору даних
X = df.drop(['ID', 'Dt_Customer', 'Education', 'Marital_Status', 'Marital_Status_clean'], axis=1)
X.reset_index(drop=True, inplace=True)

### Завдання 1: Виконання кластеризації та пониження розмірності для візуалізації результатів

Ваше завдання — провести кластеризацію клієнтів та візуалізувати результати кластеризації, використовуючи метод головних компонент (PCA) для пониження розмірності даних.

#### Інструкції:

1. **Вибір ключових характеристик:**
   Давайте обмежимось тільки наступними хараткеристиками для кластеризації цього разу:
   - `Income`: Річний дохід домогосподарства клієнта
   - `Recency`: Кількість днів з моменту останньої покупки клієнта
   - `NumStorePurchases`: Кількість покупок, зроблених безпосередньо у магазинах
   - `NumDealsPurchases`: Кількість покупок, зроблених з використанням знижок
   - `days_lifetime`: Кількість днів з моменту реєстрації клієнта у компанії
   - `years_customer`: Вік клієнта
   - `NumWebVisitsMonth`: Кількість відвідувань вебсайту компанії за останній місяць
   Відберіть в наборі даних `X` лише ці характеристики.

2. **Нормалізація даних:**
   Використайте метод `MinMaxScaler` для нормалізації значень обраних характеристик.

3. **Кластеризація:**
   Проведіть кластеризацію клієнтів, використовуючи метод `KMeans` з трьома кластерами.

4. **Пониження розмірності:**
   Використайте метод головних компонент (PCA) для пониження розмірності даних до трьох компонент.

5. **Візуалізація результатів:**
   Використовуючи plolty express побудуйте 3D-графік розподілу клієнтів у просторі трьох головних компонент, де кольором позначено кластери.

6. **Опишіть, що спостерігаєте:**
   Чи кластеризація чітко розділила дані?

Далі ми детальніше проінтерпретуємо результати візуалізації і пониження розмірностей.

In [2]:
features = ['Income', 'Recency', 'NumStorePurchases', 'NumDealsPurchases', 'days_lifetime', 'years_customer', 'NumWebVisitsMonth']

X_f = X[features]

In [3]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [4]:
X_f

,Income,Recency,NumStorePurchases,NumDealsPurchases,days_lifetime,years_customer,NumWebVisitsMonth
0,58138,58,4,3,663,57,7
1,46344,38,2,2,113,60,5
2,71613,26,10,1,312,49,4
3,26646,26,4,2,139,30,6
4,58293,94,6,5,161,33,5
...,...,...,...,...,...,...,...
2234,61223,46,4,2,381,47,5
2235,64014,56,5,7,19,68,7
2236,56981,91,13,1,155,33,6
2237,69245,8,10,2,156,58,3


In [5]:
scaled_X = pd.DataFrame(MinMaxScaler().fit_transform(X_f), columns=X_f.columns)

In [6]:
scaled_X

,Income,Recency,NumStorePurchases,NumDealsPurchases,days_lifetime,years_customer,NumWebVisitsMonth
0,0.358003,0.585859,0.307692,0.200000,0.948498,0.378641,0.35
1,0.285379,0.383838,0.153846,0.133333,0.161660,0.407767,0.25
2,0.440978,0.262626,0.769231,0.066667,0.446352,0.300971,0.20
3,0.164085,0.262626,0.307692,0.133333,0.198856,0.116505,0.30
4,0.358958,0.949495,0.461538,0.333333,0.230329,0.145631,0.25
...,...,...,...,...,...,...,...
2234,0.377000,0.464646,0.307692,0.133333,0.545064,0.281553,0.25
2235,0.394186,0.565657,0.384615,0.466667,0.027182,0.485437,0.35
2236,0.350879,0.919192,1.000000,0.066667,0.221745,0.145631,0.30
2237,0.426397,0.080808,0.769231,0.133333,0.223176,0.388350,0.15


In [7]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, n_init='auto')
kmeans.fit(scaled_X)

labels = kmeans.predict(scaled_X)
print(labels)

[0 1 2 ... 2 2 1]


In [8]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca.fit(scaled_X)

PCA(n_components=3)

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
px.scatter_3d(scaled_X, x='Income', y='Recency',z='NumStorePurchases', color=labels)

Тут спостерігається 1 чіткий кластер і два змішані

Тут та ж сама ситуація, 1 чіткий кластер і 2 змішані

In [10]:
px.scatter_3d(scaled_X, x='Income', y='days_lifetime',z='NumWebVisitsMonth', color=labels)

Тут є 2 чіткі кластери, які практично не пересікаються і мають межу між собою і елементи третього присутні у межах першого і другого кластерів

### Завдання 2: Аналіз результатів пониження розмірності

1. **Розрахунок частки поясненої дисперсії:**
   Визначте, яка частка загальної варіації даних пояснюється кожною з трьох головних компонент (PC1, PC2, PC3) за допомогою атрибуту `explained_variance_ratio_` об'єкта PCA. Виведіть результат на екран.

2. **Розрахунок кумулятивної частки поясненої дисперсії:**
   Обчисліть кумулятивну частку поясненої дисперсії для трьох головних компонент, щоб зрозуміти, скільки варіації даних пояснюється першими кількома компонентами.

In [11]:
np.sum(pca.explained_variance_ratio_)

0.8399120325811185

In [12]:
explained_variance_ratio = pca.explained_variance_ratio_

print(f"PC1 explained variance ratio: {explained_variance_ratio[0]:.4f}")
print(f"PC2 explained variance ratio: {explained_variance_ratio[1]:.4f}")
print(f"PC3 explained variance ratio: {explained_variance_ratio[2]:.4f}")

PC1 explained variance ratio: 0.3020
PC2 explained variance ratio: 0.2867
PC3 explained variance ratio: 0.2512


Судячи з отриманого результату, перша компонента пояснює 30,2% загальної варіації в даних

Друга і треті компоненти несуть в собі пояснення все ще значної частини даних 28,367% та 25,12% відповідно.

В Сумі 3 компоненти описують 83,99% варіацій даних.

In [13]:
# Обчислення кумулятивної частки поясненої дисперсії
cumulative_explained_variance = np.cumsum(explained_variance_ratio)

# Вивід кумулятивних часток для кожної компоненти
print(f"Cumulative explained variance after PC1: {cumulative_explained_variance[0]:.4f}")
print(f"Cumulative explained variance after PC2: {cumulative_explained_variance[1]:.4f}")
print(f"Cumulative explained variance after PC3: {cumulative_explained_variance[2]:.4f}")


Cumulative explained variance after PC1: 0.3020
Cumulative explained variance after PC2: 0.5887
Cumulative explained variance after PC3: 0.8399


### Завдання 3: Інтерпретація "Loadings"

Продовжуємо інтерпретацію результатів `PCA`і познайомимось з новим поняттям `loadings`, яке допоможе нам знайти звʼязок між головними компонентами і оригінальними ознаками в наборі даних.

Ми зараз побудували візуалізацію кластерів точок даних в просторі трьох головних компонент. Але хочеться знайти звʼязок між головними компонентами і оригінальними ознаками. Для розуміння, які початкові характеристики даних мають найбільший вплив на ці головні компоненти, ми можемо використати атрибут `components_` методу `PCA`.

#### Що таке `pca.components_`?

`pca.components_` — це масив, який містить коефіцієнти (або "ваги"), що показують внесок кожної вихідної ознаки у кожну з головних компонент. Ці коефіцієнти ще називаються **"loading"** або "навантаженнями" компонент.

- **Loadings** (`навантаження`) відображають важливість кожної змінної (ознаки) для відповідної головної компоненти. Вони показують, яким чином змінні поєднуються, щоб утворити нові, зменшені вимірювання.
- Якщо коефіцієнт має високе абсолютне значення (як позитивне, так і негативне), це вказує на те, що відповідна змінна сильно впливає на головну компоненту.

#### Саме завдання
Ваше завдання — обчислити "навантаження" для кожної з головних компонент і інтерпретувати результати.

1. **Обчислення loadings для компонент:**
   Використайте атрибут `components_` об'єкта PCA для створення DataFrame, який відображатиме внесок кожної вихідної ознаки в кожну головну компоненту.

2. **Інтерпретація результатів:**
   Виведіть значення "навантажень" і проаналізуйте, які ознаки найбільше впливають на кожну головну компоненту.

In [14]:
features = scaled_X.columns

loadings = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2', 'PC3'], index=features)

print(loadings)

                        PC1       PC2       PC3
Income             0.063557  0.047082 -0.373826
Recency            0.475786 -0.878876 -0.029910
NumStorePurchases  0.284282  0.187195 -0.816668
NumDealsPurchases  0.103049  0.059324  0.050687
days_lifetime      0.821912  0.431061  0.305490
years_customer     0.012658 -0.001022 -0.080411
NumWebVisitsMonth  0.049530  0.031377  0.300089


На першу компоненту найдільший вплив має days_lifetime та Recency

На другу компоненту найбільший вплив має Recency, чим і зменшує чим і зменшує значення PC2, позитивний вплив має days_lifetime

На третю компоненту позитивно впливають days_lifetime та NumWebVisitsMonth, та більш негативний вплив мають NumStorePurchases  та Income





###Завдання 4
Давайте проаналізуємо "навантаження" (**loadings**) для трьох головних компонент після вилучення ознаки `Income`. Це допоможе нам зрозуміти, як змінилася важливість інших ознак для кожної головної компоненти, коли одна з ключових характеристик (`Income`) була вилучена.

#### Кроки для проведення аналізу і ваше завдання:

1. Видаліть ознаку `Income` з нашого набору даних `X` і повторно виконайте PCA (метод головних компонент) для отримання нових "навантажень".

2. Обчисліть нові "навантаження" для трьох головних компонент на наборі даних без `Income`

3. Проаналізуйте, які ознаки мають найбільший вплив на кожну головну компоненту після вилучення `Income`.

4. Перегляньте, наскільки кожна з головних компонент пояснює дисперсію в даних без ознаки `Income`.

In [15]:
X_without_income = X_f.drop('Income', axis=1)


In [16]:
scaled_X_wi = pd.DataFrame(MinMaxScaler().fit_transform(X_without_income), columns=X_without_income.columns)
scaled_X_wi

,Recency,NumStorePurchases,NumDealsPurchases,days_lifetime,years_customer,NumWebVisitsMonth
0,0.585859,0.307692,0.200000,0.948498,0.378641,0.35
1,0.383838,0.153846,0.133333,0.161660,0.407767,0.25
2,0.262626,0.769231,0.066667,0.446352,0.300971,0.20
3,0.262626,0.307692,0.133333,0.198856,0.116505,0.30
4,0.949495,0.461538,0.333333,0.230329,0.145631,0.25
...,...,...,...,...,...,...
2234,0.464646,0.307692,0.133333,0.545064,0.281553,0.25
2235,0.565657,0.384615,0.466667,0.027182,0.485437,0.35
2236,0.919192,1.000000,0.066667,0.221745,0.145631,0.30
2237,0.080808,0.769231,0.133333,0.223176,0.388350,0.15


In [17]:
pca = PCA(n_components=3)
pca.fit(scaled_X_wi)

PCA(n_components=3)

In [18]:
features = scaled_X_wi.columns

loadings = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2', 'PC3'], index=features)

print(loadings)

                        PC1       PC2       PC3
Recency            0.500712 -0.865392 -0.008648
NumStorePurchases  0.226953  0.145537 -0.924549
NumDealsPurchases  0.105012  0.065833  0.027582
days_lifetime      0.825793  0.472036  0.232275
years_customer     0.007110 -0.005913 -0.079808
NumWebVisitsMonth  0.069051  0.052233  0.289919


На першу компоненту найбільший вплив все ще має days_lifetime, він лише трошки збільшивсята Recency піднявся трошки, а NumStorePurchases досить суттєво знизився.

На другу компоненту найбільший вплив знову має Recency, чим і зменшує чим і зменшує значення PC2, позитивний вплив має days_lifetime, трошки збільшився

На третю компоненту позитивно впливають days_lifetime та NumWebVisitsMonth, їхній вплив зменшився після видалення Income. Більш негативний вплив мають NumStorePurchases(збільшився), напевно після видалення Income перерозподілився вплив. На РС3 Income впливав суттєва.

### Завдання 5: Візуалізація кластеризації за допомогою t-SNE

Ваше завдання — використати метод t-SNE для візуалізації результатів кластеризації клієнтів у двовимірному просторі. Метод t-SNE допомагає знизити розмірність даних та зберегти локальні структури в даних, що робить його ефективним для візуалізації високорозмірних даних. Ми також зможемо порівняти результат цього методу з РСА.

1. Використайте метод t-SNE для зниження розмірності до 2х вимірів даних, які включають ознаки всі, що і в завданні 1, а також були відмасштабовані перед пониженням розмірностей.

2. Створіть новий DataFrame з координатами, отриманими після застосування t-SNE, та додайте до нього мітки кластерів.

3. Побудуйте інтерактивний 2D-графік розподілу клієнтів, де кольором буде позначено різні кластери і проаналізуйте графік з рекомендаціями нижче (можливо треба буде вивести додаткові візуалізації чи таблиці для інтерпретації, але треба прям зрозуміти, які ознаки формують який кластер і чим кластери відрізняються одне від одного).

  **Опишіть отримані кластери з точки зору ознак.**

4. Опишіть відмінність графіка tSNE від PCA.

#### ЯК можна інтерпретувати з t-SNE?

Хоча t-SNE не надає "компонентів" як РСА, він забезпечує низьковимірне представлення даних, яке можна візуально інтерпретувати:

- **Кластери:** t-SNE особливо добре показує кластери подібних точок. Якщо ви бачите чітко визначені кластери на графіку t-SNE, це свідчить про наявність груп схожих спостережень у ваших даних. Проаналізувати їх можемо, якщо додамо дані в `hover_data` або якщо якісь з даних виведемо як розмір чи форма точок на візуалізації. Також корисно може бути вивести середні значення ознак по кластерам.
- **Локальна структура:** Відносне розташування точок одного кластеру на графіку t-SNE може допомогти вам зрозуміти, які дані подібні між собою.
- **Глобальна структура:** Будьте обережні; t-SNE менш надійний для відображення глобальних структур (наприклад, відстаней між кластерами) у порівнянні з PCA, бо t-SNE націлений на збереження саме локальних структур.

In [19]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(scaled_X)

In [20]:
kmeans = KMeans(n_clusters=3, random_state=42)
cluster_labels = kmeans.fit_predict(scaled_X)

# Зниження розмірності за допомогою t-SNE
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(scaled_X)

# Створення нового датафрейму з координатами t-SNE та мітками кластера
df_tsne = pd.DataFrame(X_tsne, columns=['Component 1', 'Component 2'])
df_tsne['Cluster Label'] = cluster_labels

# Перегляд результату
print(df_tsne.head())

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



   Component 1  Component 2  Cluster Label
0     7.656736    31.979853              0
1   -42.249790    -9.923389              2
2    -5.909520    13.197768              2
3   -49.364185    -2.457292              2
4    -5.971073   -30.051323              1


In [21]:
fig = px.scatter(
    df_tsne,
    x='Component 1',
    y='Component 2',
    color='Cluster Label',
    title='t-SNE Visualization of Customer Clusters',
    labels={'Component 1': 't-SNE Component 1', 'Component 2': 't-SNE Component 2'},
    template='plotly_dark',
    size_max=30,
    width=750,
)

fig.show()

t-SNE досить гарно кластеризував.
є 3 кластери, жовтий кластер поділений ніби як на 2 частини, можливо варто на 4 кластери зробити. Кластери мають як щільні місця так і розріджені

Синій кластер має свою зону і трохи "викидів", які лежать у зоні рожевого та трошки у зоні жовтого. Теж умвоно ділиться на 2 частини. Можливо ці включення є помилкою у даних.

Візуально, якщо не брати до уваги кольори то кластерів я бачу 5, але це може бути особливість відображення t-SNE - неточність у дистанціях та більша увага до локальності.